Organization:
- Work
  - 1 test: defining functions for part 1, testing on test input
  - 1 run: getting answer for part 1
  - 2 test: ...
  - 2 run: ...
- Utilities: functions I think might help parse general inputs
- Inputs: where I define the test (_t_) and problem (_s_) inputs

# Work

## 1 test

Idea: step through time, updating the accessible positions as a set until it reaches the exit
- Intialize: grid of the blizzard locations and my set of locations is the initial location
- Step until my set of locations reaches the end: update the blizzard locations, and make the set of locations equal to all non-blizzard locations <= 1 away from the current set of locations

(Also, manual check says there are no blizzards that would have to interact with the start/end areas so no need to worry about how that might work)

For the blizzards, if there are multiple on the same square I'll just make a string out of their characters

Why this "set of accessible locations" idea makes sense here: if you draw out all the possible paths we can walk, there will (probably) be a LOT of overlap. So rather than treat each path as its own thing, we can work with the set of locations instead to take advantage of the amount of overlap.

In [8]:
import numpy as np

In [233]:
def take_step():
    global grid, locations, n, m, blank_grid
    
    # Update the blizzards
    new_grid = blank_grid.copy()
    
    # Iterate over locations
    for i in range(1,n-1):
        for j in range(1,m-1):
            # For each blizzard in this location, update each one
            for blizzard in grid[i,j]:
                loc = move_blizzard(i,j,blizzard)
                new_grid[loc] += blizzard
                
    grid = new_grid
    
    # Update our possible locations
    new_locations = locations.copy()
    
    # The main area
    for i in range(1,n-1):
        for j in range(1,m-1):
            # If we could have been here, mark that we can move to its neighbors
            if locations[i,j]:
                new_locations[i+1,j] = True
                new_locations[i-1,j] = True
                new_locations[i,j+1] = True
                new_locations[i,j-1] = True
    
    # Don't forget ones moved to from the start and end locations!
    if locations[0,1]:
        new_locations[1,1] = True
    if locations[n-1,m-2]:
        new_locations[n-2,m-2] = True
    
    # Remember we can't go into the walls (special treatment for the start and end)
    start = new_locations[0,1]
    end = new_locations[n-1,m-2]
    new_locations[0,:] = False
    new_locations[n-1,:] = False
    new_locations[:,0] = False
    new_locations[:,m-1] = False
    new_locations[0,1] = start
    new_locations[n-1,m-2] = end
    
    # Now remove the locations that have blizzards
    for i in range(1,n-1):
        for j in range(1,m-1):
            if grid[i,j]:
                new_locations[i,j] = False
    
    locations = new_locations
    

def move_blizzard(i,j,blizzard):
    global n,m
    
    # Move the blizzard
    match blizzard:
        case '>':
            di,dj = 0,1
        case 'v':
            di,dj = 1,0
        case '<':
            di,dj = 0,-1
        case '^':
            di,dj = -1,0
    i,j = i+di,j+dj
    
    # Conservation of blizzards
    if i == 0:
        i = n-2
    if i == n-1:
        i = 1
    if j == 0:
        j = m-2
    if j == m-1:
        j = 1
    
    return (i,j)

In [234]:
# Initialize the grid, replacing '.' with '' for easy blizzard management later
spl = split(t)
n = len(spl)
m = len(spl[0])
grid = np.full((n,m),'',dtype=object)
for i,line in enumerate(spl):
    for j,char in enumerate(line):
        if char != '.':
            grid[i,j] = char
grid

array([['#', '', '#', '#', '#', '#', '#', '#'],
       ['#', '>', '>', '', '<', '^', '<', '#'],
       ['#', '', '<', '', '', '<', '<', '#'],
       ['#', '>', 'v', '', '>', '<', '>', '#'],
       ['#', '<', '^', 'v', '^', '^', '>', '#'],
       ['#', '#', '#', '#', '#', '#', '', '#']], dtype=object)

In [235]:
# A blank grid
blank_grid = grid.copy()
for i in range(1,n-1):
    for j in range(1,m-1):
        blank_grid[i,j] = ''

In [236]:
# Initialize an array with my possible locations
locations = np.full((n,m),False)
locations[0,1] = True
locations

array([[False,  True, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False]])

In [237]:
# Target location
loc_target = (n-1, m-2)

In [238]:
# Iterate until we reach the end
for step in range(20):
    take_step()
    if locations[loc_target]:
        print(step+1)
        break

18


## 1 run

In [239]:
# Initialize the grid, replacing '.' with '' for easy blizzard management later
spl = split(s)
n = len(spl)
m = len(spl[0])
grid = np.full((n,m),'',dtype=object)
for i,line in enumerate(spl):
    for j,char in enumerate(line):
        if char != '.':
            grid[i,j] = char

# A blank grid
blank_grid = grid.copy()
for i in range(1,n-1):
    for j in range(1,m-1):
        blank_grid[i,j] = ''

# Initialize an array with my possible locations
locations = np.full((n,m),False)
locations[0,1] = True

# Target location
loc_target = (n-1, m-2)

In [240]:
# Iterate until we reach the end
for step in range(1000):
    take_step()
    if locations[loc_target]:
        print(step+1)
        break

269


## 2 test

Just run part 1 three times. I went back and modified the code to not rely on the start/end location, so that I just have to appropriately define my "locations" array to get things to run end to start.

In [251]:
# Setup

# Initialize the grid, replacing '.' with '' for easy blizzard management later
spl = split(t)
n = len(spl)
m = len(spl[0])
grid = np.full((n,m),'',dtype=object)
for i,line in enumerate(spl):
    for j,char in enumerate(line):
        if char != '.':
            grid[i,j] = char

# A blank grid
blank_grid = grid.copy()
for i in range(1,n-1):
    for j in range(1,m-1):
        blank_grid[i,j] = ''

In [252]:
# There

# Initialize an array with my possible locations
locations = np.full((n,m),False)
locations[0,1] = True

# Target location
loc_target = (n-1, m-2)

# Iterate until we reach the end
for step in range(1000):
    take_step()
    if locations[loc_target]:
        print(step+1)
        break

18


In [254]:
# Back

# Initialize an array with my possible locations
locations = np.full((n,m),False)
locations[n-1,m-2] = True

# Target location
loc_target = (0,1)

# Iterate until we reach the end
for step in range(1000):
    take_step()
    if locations[loc_target]:
        print(step+1)
        break

23


In [255]:
# Back

# Initialize an array with my possible locations
locations = np.full((n,m),False)
locations[0,1] = True

# Target location
loc_target = (n-1,m-2)

# Iterate until we reach the end
for step in range(1000):
    take_step()
    if locations[loc_target]:
        print(step+1)
        break

13


In [256]:
18+23+13

54

## 2 run

In [257]:
# Setup

# Initialize the grid, replacing '.' with '' for easy blizzard management later
spl = split(s)
n = len(spl)
m = len(spl[0])
grid = np.full((n,m),'',dtype=object)
for i,line in enumerate(spl):
    for j,char in enumerate(line):
        if char != '.':
            grid[i,j] = char

# A blank grid
blank_grid = grid.copy()
for i in range(1,n-1):
    for j in range(1,m-1):
        blank_grid[i,j] = ''

In [258]:
# There

# Initialize an array with my possible locations
locations = np.full((n,m),False)
locations[0,1] = True

# Target location
loc_target = (n-1, m-2)

# Iterate until we reach the end
for step in range(1000):
    take_step()
    if locations[loc_target]:
        print(step+1)
        break

269


In [259]:
# Back

# Initialize an array with my possible locations
locations = np.full((n,m),False)
locations[n-1,m-2] = True

# Target location
loc_target = (0,1)

# Iterate until we reach the end
for step in range(1000):
    take_step()
    if locations[loc_target]:
        print(step+1)
        break

286


In [260]:
# Back

# Initialize an array with my possible locations
locations = np.full((n,m),False)
locations[0,1] = True

# Target location
loc_target = (n-1,m-2)

# Iterate until we reach the end
for step in range(1000):
    take_step()
    if locations[loc_target]:
        print(step+1)
        break

270


In [261]:
269 + 286 + 270

825

# Utilities

In [4]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s, block_char = '\n\n', line_char = '\n'):
    out = [block.split(line_char) for block in clean(s).split(block_char)]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

# Inputs

In [5]:
t = """
#.######
...
######.#
"""

In [6]:
s = """
#.########################################################################################################################
...
########################################################################################################################.#
"""